In [1]:
# Install our (local, for now) pip package 
!pip3 install -e /home/jjhenkel/find-tune

In [1]:
import mlos

mlos.set_project_dir('/home/jjhenkel/sqlite/.mlos')
mlos.checkenv(using_azure=True)

project = (
    mlos
    .create_project_from_git_info()
    .with_backend(mlos.backends().azure())
    .with_build_options(mlos.build_options())
)

project.attach(db_only=True)

- Verifying environment...
   ✓ Environment verification complete
- Creating project from current git repository...
   - Getting project name: 
      ✓ project-name:='https://github.com/jjhenkel/sqlite'
   - Getting project commit hash: 
      ✓ project-commit:='06da130bcdaca675cf504b64911ade1d569d696f'
- Ensuring infrastructure is allocated...
   ✓ Found resource group 'FindTune-EUS'
   ✓ Using existing vnet: 'findtune-az-autogen-vnet-801663' (['10.0.0.0/16'])
   ✓ Using existing subnet: 'findtune-az-autogen-subnet-806943' (10.0.0.0/24)
   ✓ Using existing ip: 'findtune-az-autogen-ip-ft-db-211043' (52.226.50.179)
   ✓ Using existing vm: 'findtune-az-autogen-vm-ft-db-500381'
- Attaching...
   ✓ 'findtune-az-autogen-vm-ft-db-500381' is running!
   ✓ Skipping DB create. Database 'targets-sqlite' already exists!
   ✓ Attached. Ready to run queries


In [2]:
project.ingest()

- Preparing target docker image...
   ✓ Overriding plugin with '/home/jjhenkel/mount/velveeta/find-tune/frontends/clang/src/libDumpDatalog.so'
   - Building image findtune/mlos-target:ad645ae35e5f2520aaecaa2eda0b52fbeef08453...
      ✓ Image build complete!
      ✓ Push complete!
   ✓ Override cleanup complete!
- Running remote build...
   ✓ Build started! Use poll_status() to watch build progress.


In [3]:
project.poll_status()

✓ Build container finished --- waiting for completed sessions backlog to clear. 
✓ Completed sessions backlog cleared --- waiting for embeddings to be computed. 
✓ Ingest complete! Use project.detach(keep_db=True) and/or project.deallocate(keep_db=True) to cleanup build & gpu VMs. 


In [4]:
project.detach(keep_db=True)

- Detaching...
   ✓ 'findtune-az-autogen-vm-ft-embed-649821' is stopped!
   ✓ 'findtune-az-autogen-vm-ft-build-172657' is stopped!
   ✓ Detached. DB VM still attached. Call project.detach(keep_db=False) to detach.


In [6]:
results = (project
  .query()
  .records()
  .name_contains("config")
  .make_visualizable()
  .fields()
  .select_usages()
  .from_files_matching('/sqlite3.c')
  .execute()
).results

USES_GLOBAL = float(len([ 
  x for x in results if 'Global' in x['clang_source_range_as_text']
]))

TOTAL = float(len(results))

print("Uses sqlite3GLOBAL... in {}/{} ({:.2%}) of places".format(
  USES_GLOBAL, TOTAL, USES_GLOBAL / TOTAL
))

Uses sqlite3GLOBAL... in 159.0/167.0 (95.21%) of places


In [16]:
(project
  .query()
  .records()
  .name_contains("cache")
  .field_references()
  .restrict_to_assignments(
    where_rhs=mlos.filters().arithmetic()
  )
  .make_visualizable()
  .select()
  .execute()
  .display()
)

,gid,start_line,start_column,end_line,end_column,file,clang_source_range_as_text
0,-1878978231921796428,50584,5,50584,37,/sqlite3.c,pCache->n90pct = pCache->nMax*9/10
1,1895907294033749948,50914,5,50914,30,/sqlite3.c,pCache->iMaxKey = iLimit-1


In [3]:
(
  project
    .query()
    .records()
    .comment_semantically_similar_to("page cache")
    .make_visualizable()
    .select()
    .execute()
    .display()
)

In [4]:
(project
  .query()
  .records()
  .fields()
  .integers()
  .comment_semantically_similar_to(["desired", "size"])
  .make_visualizable()
  .select()
  .execute()
  .display()
)